# taq_cross_response_day_time_shift_data

#### Juan Camilo Henao Londono - 26.06.2019
#### AG Guhr - Universitaet Duisburg-Essen

In the following [link]() can be seen the plot of the function's results

In [1]:
# Modules

import numpy as np
import os

import pickle

import taq_data_tools

__tau__ = 1000

In [2]:
def taq_cross_response_day_time_shift_data(ticker_i, ticker_j, date, tau):
    """
    Obtain the cross response function using the midpoint price returns of
    ticker i and trade signs of ticker j during different time lags. The data
    is adjusted to use only the values each second. Return an array with the
    cross response function for a day.
        :param ticker_i: string of the abbreviation of the midpoint stock to
         be analized (i.e. 'AAPL')
        :param ticker_j: string of the abbreviation of the trade sign stock to
         be analized (i.e. 'AAPL')
        :param date: string with the date of the data to be extracted
         (i.e. '2008-01-02')
    """

    date_sep = date.split('-')

    year = date_sep[0]
    month = date_sep[1]
    day = date_sep[2]

    if (ticker_i == ticker_j):

        # Self-response

        return None

    else:

        try:

            function_name = taq_cross_response_day_time_shift_data.__name__
            taq_data_tools.taq_function_header_print_data(function_name,
                                                          ticker_i, ticker_j,
                                                          year, month, day)

            # Load data
            midpoint_i = pickle.load(open(''.join((
                    '../../taq_data/article_reproduction_data_{1}/taq'
                    + '_midpoint_full_time_data/taq_midpoint_full_time_data'
                    + '_midpoint_{1}{2}{3}_{0}.pickle').split())
                    .format(ticker_i, year, month, day), 'rb'))
            trade_sign_j = pickle.load(open("".join((
                    '../../taq_data/article_reproduction_data_2008/taq_trade_'
                    + 'signs_full_time_data/taq_trade_signs_full_time_data'
                    + '_{1}{2}{3}_{0}.pickle').split())
                    .format(ticker_j, year, month, day), 'rb'))
            # As the data is loaded from the original reproduction data from
            # the article, the data have a shift of 1 second. To correct this
            # I made both data to have the same time [34801, 56999]
            midpoint_i = midpoint_i[1:]
            trade_sign_j = trade_sign_j[:-1]

            assert len(midpoint_i) == len(trade_sign_j)

            # Array of the average of each tau. 10^3 s used by Wang
            shift_val = range(- 10 * tau, 10 * tau, tau)
            cross_response_shift = np.zeros(len(shift_val))
            num = np.zeros(len(shift_val))

            # Calculating the midpoint return and the cross response function

            # Depending on the tau value
            for s_idx, s_val in enumerate(shift_val):

                if (s_val < 0):
                    midpoint_shift = midpoint_i[np.abs(s_val):]
                    trade_sign_shift = trade_sign_j[:-np.abs(s_val)]

                elif (s_val > 0):
                    midpoint_shift = midpoint_i[:-s_val]
                    trade_sign_shift = trade_sign_j[s_val:]

                else:
                    midpoint_shift = midpoint_i
                    trade_sign_shift = trade_sign_j

                trade_sign_tau = 1 * trade_sign_shift[:-tau - 1]
                trade_sign_no_0_len = len(trade_sign_tau[trade_sign_tau != 0])
                num[s_idx] = trade_sign_no_0_len
                # Obtain the midpoint log return. Displace the numerator tau
                # values to the right and compute the return

                # Midpoint price returns

                log_return_i_sec = (midpoint_shift[tau + 1:]
                                    - midpoint_shift[:-tau - 1]) \
                    / midpoint_shift[:-tau - 1]

                # Obtain the cross response value
                if (trade_sign_no_0_len != 0):
                    product = log_return_i_sec * trade_sign_tau
                    cross_response_shift[s_idx] = np.sum(product)

            return cross_response_shift, num

        except FileNotFoundError:
            print('No data')
            print()
            return None

In [3]:
# Test

ticker_i = 'AAPL'
ticker_j =  'MSFT'
dates = []

year = '2008'
month = '03'
days = ['10', '11', '12', '13', '14']

tau = 1

for day in days:
    dates.append(year + '-' + month + '-' + day)

for date in dates:
    taq_cross_response_day_time_shift_data(ticker_i, ticker_j, date, tau)

TAQ data
taq_cross_response_day_time_shift_data
Processing data for the stock i AAPL and stock j MSFT the 2008.03.10
TAQ data
taq_cross_response_day_time_shift_data
Processing data for the stock i AAPL and stock j MSFT the 2008.03.11
TAQ data
taq_cross_response_day_time_shift_data
Processing data for the stock i AAPL and stock j MSFT the 2008.03.12
TAQ data
taq_cross_response_day_time_shift_data
Processing data for the stock i AAPL and stock j MSFT the 2008.03.13
TAQ data
taq_cross_response_day_time_shift_data
Processing data for the stock i AAPL and stock j MSFT the 2008.03.14
